In [1]:
# Load packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import f1_score
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


In [2]:
# Load data

predict1 = pd.read_csv('../reports/ion_submission_WaveNet3_2.2.2.csv')
predict2 = pd.read_csv('../reports/ion_submission_WaveNet4_2.2.2.csv')
predict3 = pd.read_csv('../reports/ion_submission_catclass3_2.1.7.csv') #RF stacked
predict4 = pd.read_csv('../reports/ion_submission_catclass2_2.1.9.csv') #high catpred
predict5 = pd.read_csv('../reports/ion_submission_catclass1_2.1.7.1.csv')#new sqroot


In [3]:
all_predictions = pd.concat([predict1, predict2['open_channels'], predict3['open_channels'], predict4['open_channels'], predict5['open_channels']], axis = 1)

# Trying to find which models have most diagreement

In [4]:
all_predictions['is_equal1'] = (all_predictions.iloc[:,1] == all_predictions.iloc[:,2]) 
all_predictions['is_equal2'] = all_predictions.iloc[:,1] == all_predictions.iloc[:,3] 
all_predictions['is_equal3'] = all_predictions.iloc[:,1]==  all_predictions.iloc[:,4] 
all_predictions['is_equal4'] =all_predictions.iloc[:,1] == all_predictions.iloc[:,5]

In [5]:
all_predictions['is_equal'] = all_predictions['is_equal1'] |all_predictions['is_equal2'] |all_predictions['is_equal3'] |all_predictions['is_equal4'] 

In [6]:
different = all_predictions[all_predictions.is_equal1 != True]
different.shape

(1192, 11)

In [7]:
different.iloc[0:50,:]

,time,open_channels,open_channels,open_channels,open_channels,open_channels,is_equal1,is_equal2,is_equal3,is_equal4,is_equal
9317,500.9318,1,0,1,0,0,False,True,False,False,True
35306,503.5307,1,0,0,0,1,False,False,False,True,True
44657,504.4658,1,0,1,0,1,False,True,False,True,True
91023,509.1024,1,0,0,0,0,False,False,False,False,False
92258,509.2259,1,0,0,0,0,False,False,False,False,False
99998,509.9999,1,0,0,1,1,False,False,True,True,True
100010,510.0011,2,3,2,2,3,False,True,True,False,True
100713,510.0714,1,0,0,0,0,False,False,False,False,False
100895,510.0896,1,2,2,2,2,False,False,False,False,False
100959,510.0960,1,2,2,2,2,False,False,False,False,False


# Average predictions from the 5 models

In [8]:
average = (all_predictions.iloc[:,1] + all_predictions.iloc[:,2] + all_predictions.iloc[:,3] + all_predictions.iloc[:,4] +all_predictions.iloc[:,5])/5

In [9]:
all_predictions['average'] = np.round(average, 0).astype(int)

In [10]:
test= pd.read_csv('../data/raw/test.csv')
test['open_channels'] = all_predictions['average']

In [11]:
test[['time','open_channels']].to_csv('../reports/average.csv', index=False, float_format='%.4f')

In [12]:
test.head()

,time,signal,open_channels
0,500.0001,-2.6498,0
1,500.0002,-2.8494,0
2,500.0003,-2.8600,0
3,500.0004,-2.4350,0
4,500.0005,-2.6155,0


In [13]:
all_predictions[all_predictions.iloc[:,3] == 6].count()

time             37244
open_channels    37244
open_channels    37244
open_channels    37244
open_channels    37244
open_channels    37244
is_equal1        37244
is_equal2        37244
is_equal3        37244
is_equal4        37244
is_equal         37244
average          37244
dtype: int64

In [14]:
all_predictions[all_predictions.iloc[:,2] == 6].count()

time             37158
open_channels    37158
open_channels    37158
open_channels    37158
open_channels    37158
open_channels    37158
is_equal1        37158
is_equal2        37158
is_equal3        37158
is_equal4        37158
is_equal         37158
average          37158
dtype: int64

In [15]:
pd.DataFrame(all_predictions.iloc[1000000:, 0:2]).groupby('open_channels').count()

,time
open_channels,
0,916732
1,78182
2,4714
3,341
4,31


In [16]:
pd.concat([all_predictions.iloc[1000000:, 0], all_predictions.iloc[1000000:, 2]], axis = 1).groupby('open_channels').count()

,time
open_channels,
0,916858
1,78060
2,4709
3,341
4,32


In [17]:
pd.concat([all_predictions.iloc[1000000:, 0], all_predictions.iloc[1000000:, 3]], axis = 1).groupby('open_channels').count()

,time
open_channels,
0,916874
1,78018
2,4734
3,340
4,34
